In [2]:
%load_ext autoreload
%autoreload 2

import os
import csv
import numpy as np
import shapely
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import torch


from torch.utils.data import DataLoader, RandomSampler, ConcatDataset
from pytorch_lightning.utilities import CombinedLoader
from pathlib import Path
from datetime import datetime

import dl_toolbox.callbacks as callbacks
import dl_toolbox.modules as modules 
import dl_toolbox.networks as networks
import dl_toolbox.datasets as datasets
import dl_toolbox.torch_collate as collate
import dl_toolbox.utils as utils

import rasterio.windows as windows

torch.set_float32_matmul_precision('high')
test = False
if os.uname().nodename == 'WDTIS890Z': 
    data_root = Path('/mnt/d/pfournie/Documents/data')
    home = Path('/home/pfournie')
    save_root = data_root / 'outputs'
elif os.uname().nodename == 'qdtis056z': 
    data_root = Path('/data')
    home = Path('/d/pfournie')
    save_root = data_root / 'outputs'
elif os.uname().nodename.endswith('sis.cnes.fr'):
    home = Path('/home/eh/fournip')
    save_root = Path('/work/OT/ai4usr/fournip') / 'outputs'
    if test:
        data_root = Path('/work/OT/ai4geo/DATA/DATASETS')
    else:
        #!bash '/home/eh/fournip/dl_toolbox/copy_data_to_node.sh'
        data_root = Path(os.environ['TMPDIR'])

/d/pfournie/dl_toolbox/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# datasets params
dataset_name = 'DIGITANIE'
data_path = data_root / dataset_name
nomenclature = datasets.DigitanieNomenclatures['building'].value
num_classes=len(nomenclature)
crop_size=256
crop_step=64
bands = [1,2,3]

# split params
split = home / f'dl_toolbox/dl_toolbox/datamodules/digitanie_all.csv'

train_idx = [i+j for i in [0, 66, 88, 99, 110, 154] for j in range(1,9)]
train_aug = 'd4_color-3'

val_idx = [i+j for i in [0, 66, 88, 99, 110, 154] for j in range(9,11)]
val_aug = 'no'

unsup_idx = []
unsup_aug = 'd4'

# dataloaders params
batch_size = 16
epoch_steps = 1000
num_samples = epoch_steps * batch_size
num_workers=6

# network params
in_channels=len(bands)
out_channels=num_classes
pretrained = 'imagenet'
encoder='efficientnet-b1'

# module params
mixup=0. # incompatible with ignore_zero=True
class_weights = [1., 1., 3.] #[1.] * num_classes
initial_lr=0.001
ttas=[]
alpha_ramp=utils.SigmoidRamp(2,4,0.,0.)
pseudo_threshold=0.9
consist_aug='color-5'
ema_ramp=utils.SigmoidRamp(2,4,0.9,0.99)

# trainer params
num_epochs = 50
#max_steps=num_epochs * epoch_steps
accelerator='gpu'
devices=1
multiple_trainloader_mode='min_size'
limit_train_batches=1.
limit_val_batches=1.
save_dir = save_root / dataset_name
log_name = 'fra->Nantes'
ckpt_path=None # '/data/outputs/test_bce_resisc/version_2/checkpoints/epoch=49-step=14049.ckpt'

In [25]:
train_data_src = [
    src for src in datasets.datasets_from_csv(
        data_path,
        split,
        train_idx
    )
]

train_sets = [
    datasets.Raster2Cls(
        data_src=src,
        crop_size=crop_size,
        aug=train_aug,
        bands=bands,
        nomenclature=nomenclature
    ) for src in train_data_src
]

train_set = ConcatDataset(train_sets)

val_data_src = [
    src for src in datasets.datasets_from_csv(
        data_path,
        split,
        val_idx
    )
]

val_sets = [
    datasets.Raster2Cls(
        data_src=src,
        crop_size=crop_size,
        #crop_step=crop_size//2,
        aug=val_aug,
        bands=bands,
        nomenclature=nomenclature
    ) for src in val_data_src
]

val_set = ConcatDataset(val_sets)

if unsup_idx:
    
    train_set_coords = []
    
    for src in train_data_src:
        if isinstance(src.zone, windows.Window):
            bbox = windows.bounds(src.zone, src.meta['transform'])
            train_set_coords.append(datasets.polygon_from_bbox(bbox))
        elif isinstance(src.zone, shapely.Polygon):
            train_set_coords.append(src.zone.exterior.coords)
        
    unsup_sets = []

    unsup_data_src = datasets.datasets_from_csv(
        data_path,
        split,
        unsup_idx
    )

    for src in unsup_data_src:

        if isinstance(src.zone, windows.Window):
            bbox = windows.bounds(src.zone, src.meta['transform'])
            poly_zone = shapely.Polygon(
                shell=datasets.polygon_from_bbox(bbox),
                #holes=train_set_coords
            )

        elif isinstance(src.zone, shapely.Polygon):
            poly_zone = shapely.Polygon(
                shell=src.zone.exterior.coords,
                #holes=train_set_coords
            )

        src.zone = poly_zone

        unsup_set = datasets.Raster(
            data_src=src,
            crop_size=crop_size,
            aug=unsup_aug,
            bands=bands,
            nomenclature=nomenclature,
        )

        unsup_sets.append(unsup_set)

    unsup_set = ConcatDataset(unsup_sets) 

In [ ]:
%matplotlib inline
from shapely import plotting
from matplotlib import colors
import matplotlib.pyplot as plt
from copy import copy
from rasterio import plot as rioplt

def shape_color(name, alpha):
    color = colors.to_rgba(name)
    facecolor = list(color)
    facecolor[-1] = alpha
    facecolor = tuple(facecolor)
    edgecolor = color
    return facecolor, edgecolor

#fig, ax = plt.subplots(1, 1)
#f0, e0 = shape_color("C0", 0)
#f1, e1 = shape_color("C1", 0)
#
ds = val_set.datasets[0]
#poly_zone = ds.data_src.zone
#poly_zone = shapely.Polygon(datasets.polygon_from_bbox(windows.bounds(ds.data_src.zone, ds.data_src.meta['transform'])))
#patch_zone = plotting.patch_from_polygon(poly_zone, facecolor=f0, edgecolor=e0)
#ax.add_patch(copy(patch_zone))
#
#for coord in train_set_coords:
#    train_poly = shapely.Polygon(coord)
#    patch = plotting.patch_from_polygon(train_poly, facecolor=f1, edgecolor=e1)
#    ax.add_patch(copy(patch))
    
item =ds[1]
image=item['image'].numpy()
#print(image.min())
#rioplt.show(image, ax=ax, transform=item['crop_tf'])
#ax.axis('equal')
#ax.autoscale_view()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(5,10))
rioplt.show(image[:3,...], ax=ax1, transform=item['crop_tf'])
ax1.set_title(nomenclature[item['label'].numpy()].name)
ax2.imshow(image[3])

tensor([0.1054, 0.8734, 0.0207], dtype=torch.float64)


In [ ]:
train_dataloaders = {}
train_dataloaders['sup'] = DataLoader(
    dataset=train_set,
    batch_size=batch_size,
    collate_fn=collate.CustomCollate(),
    sampler=RandomSampler(
        data_source=train_set,
        replacement=True,
        num_samples=num_samples
    ),
    num_workers=num_workers,
    drop_last=True
)

#train_dataloaders['unsup'] = DataLoader(
#    dataset=unsup_set,
#    batch_size=batch_size,
#    collate_fn=collate.CustomCollate(),
#    sampler=RandomSampler(
#        data_source=unsup_set,
#        replacement=True,
#        num_samples=num_samples
#    ),
#    num_workers=num_workers,
#    drop_last=True
#)

val_dataloader = DataLoader(
    dataset=val_set,
    shuffle=False,
    #sampler=RandomSampler(
    #    data_source=val_set,
    #    replacement=True,
    #    num_samples=num_samples//10
    #),
    collate_fn=collate.CustomCollate(),
    batch_size=batch_size,
    num_workers=num_workers
)

In [26]:
network = networks.SmpUnet(
    encoder=encoder,
    in_channels=in_channels,
    out_channels=out_channels,
    pretrained=pretrained
)

### Building lightning module
module = modules.Supervised(
    mixup=mixup, # incompatible with ignore_zero=True
    network=network,
    num_classes=num_classes,
    class_weights=class_weights,
    initial_lr=initial_lr,
    ttas=ttas,
    #alpha_ramp=alpha_ramp,
    #pseudo_threshold=pseudo_threshold,
    #consist_aug=consist_aug,
    #ema_ramp=ema_ramp
)

In [27]:
### Metrics and plots from confmat callback
metrics_from_confmat = callbacks.MetricsFromConfmat(        
    num_classes=num_classes,
    class_names=[label.name for label in nomenclature]
)

logger = pl.loggers.TensorBoardLogger(
    save_dir=save_dir,
    name=log_name,
    version=f'{datetime.now():%d%b%y-%Hh%Mm%S}'
)

### Trainer instance
trainer = pl.Trainer(
    max_epochs=num_epochs,
    accelerator=accelerator,
    devices=devices,
    num_sanity_val_steps=0,
    limit_train_batches=limit_train_batches,
    limit_val_batches=limit_val_batches,
    logger=logger,
    callbacks=[
        pl.callbacks.ModelCheckpoint(),
        pl.callbacks.EarlyStopping(
            monitor='Val_loss',
            patience=10
        ),
        metrics_from_confmat,
        callbacks.MyProgressBar()
    ]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..


In [28]:
trainer.fit(
    model=module,
    train_dataloaders=CombinedLoader(train_dataloaders, mode=multiple_trainloader_mode),
    val_dataloaders=val_dataloader,
    ckpt_path=ckpt_path
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-d85c6e41-6024-cbb5-17b1-edd48f0192a2]

  | Name    | Type             | Params
---------------------------------------------
0 | network | SmpUnet          | 8.8 M 
1 | loss    | CrossEntropyLoss | 0     
---------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.030    Total estimated model params size (MB)


Epoch 0: 100%|██████████| 1000/1000 [02:37<00:00,  6.34it/s, v_num=5m30]
Validation: 0it [00:00, ?it/s]

/work/OT/ai4usr/fournip/latest/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 4:  44%|████▎     | 435/1000 [01:11<01:33,  6.06it/s, v_num=5m30] 